In [1]:
import os
from glob import glob
from datetime import datetime
from collections import namedtuple
from pickle import dump, load

import numpy as np
import pandas as pd
import plotly.express as px
from sklearn.preprocessing import OneHotEncoder
from sklearn.model_selection import train_test_split
from sklearn.metrics import classification_report, accuracy_score, f1_score, confusion_matrix, precision_score , recall_score
import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
from torch.utils.data import DataLoader
from torch.utils.tensorboard import SummaryWriter
import transformers
from transformers import AutoModel, AutoConfig, AutoModelForSequenceClassification, AutoTokenizer, BertTokenizer, EarlyStoppingCallback, BatchEncoding
from transformers.data.processors import SingleSentenceClassificationProcessor
from transformers import Trainer, TrainingArguments
from transformers.integrations import TensorBoardCallback
from transformers.trainer_utils import EvaluationStrategy
from transformers.data.processors.utils import InputFeatures
import optuna
from sklearn.metrics import accuracy_score


from arabert.preprocess import ArabertPreprocessor

In [2]:
model_name = "aubmindlab/bert-base-arabertv2"
tokenizer = AutoTokenizer.from_pretrained(model_name)
arabert_prep = ArabertPreprocessor(model_name)
sequence_length = 128

[2021-10-01 17:26:23,187 - farasapy_logger - WARNING]: Be careful with large lines as they may break on interactive mode. You may switch to Standalone mode for such cases.


In [3]:
def get_SMADC_folder_data():
    """Returns a dataframe with Text and Region columns. Requires tree like this data/SMADC/*.txt"""
    files = glob("data/SMADC/*.txt")
    dataframes = []

    for file in files:
        region = file[-7:-4]
        temp_df = pd.read_csv(file, encoding="utf8", delimiter="\n", names=["Text"])
        temp_df["Region"] = region
        dataframes.append(temp_df)
        
    return pd.concat(dataframes)


def get_music_df():
    files = ["GLF","LEV","NOR","IRQ"]
    dataframes = []
    
    for file in files:
        temp_df = pd.read_csv(f'../extra_data/d7_data/{file}.txt', encoding="utf8", delimiter="\n", names=["Text"])
        temp_df["Region"] = file
        dataframes.append(temp_df)
    
    return pd.concat(dataframes)


def tokenize(batch):
    """
    Tokenizes a list of strings
    """
    return tokenizer.batch_encode_plus(
        batch,
        add_special_tokens=True,
        padding="max_length",
        max_length=sequence_length,
        truncation=True,
        return_tensors="pt",
        return_attention_mask=True,
        return_token_type_ids=False,
    )


def batch_tokenize_iter(data, batch_size):
    len_data = len(data)
    batch_num = len_data // batch_size
    batch_rest = len_data / batch_size - batch_num
    
    for i in range(batch_size):
        yield tokenize(data[i * batch_num:(i+1) * batch_num].to_list())
        
    if batch_rest:
        yield tokenize(data[batch_num:].to_list())


def batch_tokenize(data, batch_size):
    bt = batch_tokenize_iter(data, batch_size)
    for i, tokenization in enumerate(bt):
        if not i:
            encoding = tokenization
            continue
        encoding["input_ids"] = torch.cat([encoding["input_ids"], tokenization["input_ids"]])
        encoding["attention_mask"] = torch.cat([encoding["attention_mask"], tokenization["attention_mask"]])
    return encoding


def preprocess_sample(sample):
    """Sample list of strings"""
    return tokenize(list(arabert_prep.preprocess(text) for text in sample))


def save_preprocessed_data(dataset, dataset_name):
    with open(f"preprocessed_data/{dataset_name}.pkl", "wb") as file:
        dump(dataset, file)
        
def load_preprocessed_data(dataset_name):
    with open(f"preprocessed_data/{dataset_name}.pkl", "rb") as file:
        temp = load(file)
    return temp


def compute_metrics(p): 
    preds = np.argmax(p.predictions, axis=1)
    assert len(preds) == len(p.label_ids)

    macro_f1 = f1_score(p.label_ids,preds,average='macro')
    macro_precision = precision_score(p.label_ids,preds,average='macro')
    macro_recall = recall_score(p.label_ids,preds,average='macro')
    acc = accuracy_score(p.label_ids,preds)
    return {
      'macro_f1' : macro_f1,
      'macro_precision': macro_precision,
      'macro_recall': macro_recall,
      'accuracy': acc
    }


def model_init(model_name, num_labels, label2id, id2label):
    return AutoModelForSequenceClassification.from_pretrained(model_name, return_dict=True, num_labels=num_labels, label2id=label2id, id2label=id2label)

In [4]:
%%time
# Dataset class
class Dialect_dataset(torch.utils.data.Dataset):
    def __init__(self, X, Y):
        super(Dialect_dataset).__init__()
        self.X = X
        self.Y = Y
        
    def __getitem__(self, key):
        return InputFeatures(self.X["input_ids"][key], self.X["attention_mask"][key], label=self.Y[key])
        
    def __len__(self):
        return len(self.X["input_ids"])

Wall time: 0 ns


In [5]:
%%time
# Date to dataframe (2.9 s)
df = get_music_df()

# Encode Y (307 ms)
df["Labels"] = df["Region"].apply({
    "EGY": 0,
    "GLF": 1,
    "IRQ": 2,
    "LEV": 3,
    "NOR": 4
  }.get)

# Preprocess X (16min 22s)
df["Text"] = df["Text"].apply(arabert_prep.preprocess)

Wall time: 37.1 s


In [6]:
%%time
df_encoding = tokenize(df["Text"].to_list())

Wall time: 3.91 s


In [7]:
test_set = Dialect_dataset(df_encoding, df["Labels"].to_list())

In [8]:
model = AutoModelForSequenceClassification.from_pretrained("models/finalized_models/2021-09-30-train-0.8921535648994515")

404 Client Error: Not Found for url: https://huggingface.co/models/finalized_models/2021-09-30-train-0.8921535648994515/resolve/main/config.json


OSError: Can't load config for 'models/finalized_models/2021-09-30-train-0.8921535648994515'. Make sure that:

- 'models/finalized_models/2021-09-30-train-0.8921535648994515' is a correct model identifier listed on 'https://huggingface.co/models'

- or 'models/finalized_models/2021-09-30-train-0.8921535648994515' is the correct path to a directory containing a config.json file



In [ ]:
trainer = Trainer(model=model,compute_metrics=compute_metrics,train_dataset=test_set)

In [ ]:
prediction = trainer.predict(test_set)

In [ ]:
prediction.metrics

In [ ]:
y_pred = prediction.label_ids
y_true = df["Labels"]
accuracy_score(y_true , y_pred)